In [236]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers
from scipy.stats import rankdata
from sklearn.metrics import r2_score


In [175]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
        "build-linux-kernel1",
        "fio2",
        "fio3",
        "fio4",
        "fio5",
        "fio6",
        "fio7",
        "fio8",
        "fio9",
        "iperf10",
        "iperf11",
        "iperf12",
        "iperf13",
        "john-the-ripper14",
        "john-the-ripper15",
        "ramspeed16",
        "ramspeed17",
        "ramspeed18",
        "ramspeed19",
        "ramspeed20",
        "ramspeed21",
        "ramspeed22",
        "ramspeed23",
        "ramspeed24",
        "ramspeed25",
        "stream26",
        "stream27",
        "stream28",
        "stream29",
        "pCpu",
        "cpus",
        "rss",
        "vmem",
        "rchar",
        "wchar",
        "syscr",
        "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                   'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                   'syscw']
x_cols = filtered_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

## Deprecated

In [ ]:
t = rTable(title="results")
t.add_column("cv")
t.add_column("unknown")
t.add_column("cv score")
t.add_column("unknown score")
for split in getSplits(dF, 1, x_cols, y_cols, cvSize=1, unknownSize=1):
    X_train, y_train, X_test, y_test, X_unknown, y_unknown, X_trans, ukwfs, cvwfs = split
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    # print(cvwfs, ukwfs, model.score(X_test, y_test), model.score(X_unknown, y_unknown))
    t.add_row(cvwfs[0], ukwfs[0], str(model.score(X_test, y_test)), str(model.score(X_unknown, y_unknown)))
rich.print(t)

In [ ]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X)))
res2 = list()
for i in range(100):
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled, y, test_size=0.2)
    model = tree.DecisionTreeRegressor()
    model.fit(X_train, y_train)
    res2.append(model.score(X_test, y_test))

In [ ]:
rich.print(np.average(res2), np.std(res2))

## Current

In [120]:
taskNames = pds.unique(dF.taskName)


def pickCVTasks():
    cvTasks = list(taskNames)
    totalData = len(dF)
    while len(dF.query("taskName in @cvTasks")) > .3 * totalData:
        cvTasks = list()
        remainingTasks = list(taskNames)
        while len(dF.query("taskName in @cvTasks")) < .2 * totalData:
            pick = np.random.choice(remainingTasks)
            remainingTasks.remove(pick)
            cvTasks.append(pick)
    return cvTasks


nodeIds = pds.unique(dF.nodeConfig)


def pickCVNodes(p=0.8):
    cvNodes = list(nodeIds)
    totalData = len(dF)
    while len(dF.query("nodeConfig in @cvNodes")) > (p + 0.05) * totalData:
        cvNodes = list()
        remainingNodes = list(nodeIds)
        while len(dF.query("nodeConfig in @cvNodes")) < p * totalData:
            pick = np.random.choice(remainingNodes)
            remainingNodes.remove(pick)
            cvNodes.append(pick)
    return cvNodes

In [23]:
def listProd(l):
    return reduce(lambda a, x: a * x, l, 1)

In [ ]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X)))
res2 = list()
done = set()
for i in range(100):
    cvNodes = pickCVNodes()
    #while listProd([i - 164 for i in cvNodes]) in done:
    #    cvNodes = pickCVNodes()
    #done.add(listProd([i - 164 for i in cvNodes]))
    train = dF.query("nodeConfig not in @cvNodes")
    X_train = train[x_cols]
    X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
    y_train = train[y_cols]
    test = dF.query("nodeConfig in @cvNodes")
    X_test = test[x_cols]
    X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
    y_test = test[y_cols]
    model = tree.DecisionTreeRegressor()
    #model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    res2.append((cvNodes, model.score(X_test, y_test)))

In [ ]:
t = rTable(title="results")
t.add_column("cv tasks")
t.add_column("score")
for r in res2[:5]:
    cv, s = r
    print(cv, s)
    #t.add_row(", ".join(list(cv)), str(s))
print("average:", np.average([s for cv, s in res2]), "std:", np.std([s for cv, s in res2]))
#rich.print(t)


In [ ]:
tmp = dict()
seen = dict()
for r in res2:
    cv, s = r
    for c in cv:
        if c not in tmp.keys():
            tmp[c] = 0
            seen[c] = 0
        tmp[c] = tmp[c] + s / len(cv)
        seen[c] = seen[c] + 1
for k in tmp.keys():
    tmp[k] = tmp[k] / seen[k]
t1 = tmp.keys()
t2 = tmp.values()
t = list(zip(t1, t2))
print(sorted(t, key=lambda x: x[1]))
print(np.average(list(t2)), np.std(list(t2)))

## Test n cv nodes

In [ ]:

nodeIds = pds.unique(dF.nodeConfig)


def pickNCVNodes(n):
    return np.random.choice(nodeIds, size=n)

In [ ]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
X_scaled = scale2.transform(poly.transform(scale1.transform(X)))
res2 = list()
done = set()
for n in range(1, len(nodeIds)):
    for i in range(min(100, len(nodeIds) ** n)):
        cvNodes = pickNCVNodes(n)
        #while listProd([i - 164 for i in cvNodes]) in done:
        #    cvNodes = pickCVNodes()
        #done.add(listProd([i - 164 for i in cvNodes]))
        train = dF.query("nodeConfig not in @cvNodes")
        X_train = train[x_cols]
        X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
        y_train = train[y_cols]
        test = dF.query("nodeConfig in @cvNodes")
        X_test = test[x_cols]
        X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
        y_test = test[y_cols]
        model = tree.DecisionTreeRegressor()
        #model = linear_model.LinearRegression()
        model.fit(X_train, y_train)
        res2.append((n, cvNodes, model.score(X_test, y_test)))
    print(n)

In [ ]:
tmp2 = dict()
for r in res2:
    n, cv, s = r
    if n not in tmp2.keys():
        tmp2[n] = list()
    tmp2[n].append(s)
tmp3 = list()
for k, v in tmp2.items():
    print(k, np.average(v), np.std(v))
    tmp3.append(np.average(v))

In [ ]:
import seaborn as sns

p = sns.lineplot(x=range(1, len(nodeIds)), y=tmp3)

## Test rank accuracy

In [237]:
X_trans = getTransformers(X, 1)
scale1, poly, scale2 = X_trans
cvNodes = pickCVNodes(.5)
train = dF.query("nodeConfig not in @cvNodes")
X_train = train[x_cols]
X_train = scale2.transform(poly.transform(scale1.transform(X_train)))
y_train = train[y_cols]
test = dF.query("nodeConfig in @cvNodes")
X_test = test[x_cols]
X_test = scale2.transform(poly.transform(scale1.transform(X_test)))
y_test = test[y_cols]
model = tree.DecisionTreeRegressor()
#model = neural_network.MLPRegressor(max_iter=1000)
#model = linear_model.LinearRegression()
model.fit(X_train, y_train)

DecisionTreeRegressor()

In [238]:
model.score(X_test, y_test)

0.8120422565459009

In [239]:
model.score(scale2.transform(poly.transform(scale1.transform(X))), y)

0.907500627863959

In [140]:
ranks = dF["rank"]

In [141]:
model.predict(scale2.transform(poly.transform(scale1.transform(X))))

array([ 3517.,  5041.,  6820., ...,  6000.,  7143., 32821.])

In [219]:
tasks = pds.unique(dF["taskName"])
wfs = pds.unique(dF["wfName"])

In [240]:
with db_actions.connect() as conn:
    test_df = pds.read_sql('SELECT * FROM "averageRuntimeScorePredBase1000"', conn)
# print(test_df)
X_test_test: pds.DataFrame = test_df[x_cols]
y_test_test: pds.DataFrame = test_df[y_cols]

X_trans_test = getTransformers(X_test_test, 1)
scale1_test, poly_test, scale2_test = X_trans_test

In [241]:
model.predict(scale2_test.transform(poly_test.transform(scale1_test.transform(X_test_test))))
t = model.predict(scale2_test.transform(poly_test.transform(scale1_test.transform(X_test_test))))
yo = test_df.assign(predTime = t)
yo = yo.sort_values(by=["wfName", "taskName", "nodeConfig"],ignore_index=True)
#print(yo)
#print(rankdata(np.array(yo.predTime).reshape((-1,27)), method="min", axis=1).reshape((-1)))
yo = yo.assign(predRank=rankdata(np.array(yo.predTime).reshape((-1,27)), method="min", axis=1).reshape((-1)))
#print(yo)
comp1 = np.array(yo["rank"]).reshape((-1,27))
comp2 = np.array(yo.predRank).reshape((-1,27))
print(r2_score(comp1, comp2))

-10.280530446496583


In [254]:
from sklearn.metrics import classification_report
print(classification_report(yo["rank"], yo.predRank))

              precision    recall  f1-score   support

           1       0.05      0.42      0.09        81
           2       0.03      0.01      0.02        86
           3       0.06      0.06      0.06        89
           4       0.05      0.04      0.04        83
           5       0.00      0.00      0.00        76
           6       0.04      0.01      0.02        75
           7       0.02      0.01      0.01        79
           8       0.04      0.01      0.02        80
           9       0.03      0.03      0.03        74
          10       0.02      0.02      0.02        83
          11       0.00      0.00      0.00        76
          12       0.03      0.03      0.03        78
          13       0.07      0.08      0.07        77
          14       0.04      0.03      0.03        79
          15       0.03      0.02      0.03        84
          16       0.06      0.12      0.09        80
          17       0.00      0.00      0.00        72
          18       0.05    

In [242]:


res3 = pds.DataFrame(
        columns=["nodeid", "task", "wf", "realtime", "pred", "predrank", "rank", "angle", "dist", "metric"])
res4 = list()
res5 = list()
for t, w in test_df[["taskName", "wfName"]].values:
    dummy1 = [t]
    dummy2 = [w]
    q = test_df.query("taskName in @dummy1 and wfName in @dummy2")
    p = model.predict(scale2_test.transform(poly_test.transform(scale1_test.transform(q[x_cols]))))
    predranks = rankdata(p, method="min")
    qo = np.array(sorted(q[["nodeConfig", "rank"]].values, key=lambda x: x[1]))[:, 0]
    po = np.array(sorted(zip(q.nodeConfig, predranks), key=lambda x: x[1]))[:, 0]
    qo2 = np.array(sorted(q[["nodeConfig", "rank"]].values, key=lambda x: x[0]))[:, 1]
    po2 = np.array(sorted(zip(q.nodeConfig, predranks), key=lambda x: x[0]))[:, 1]
    res4.append(qo2)#q["rank"])
    res5.append(po2)#predranks)
    insert = pds.DataFrame(
            data={"nodeid": q.nodeConfig, "task": q.taskName, "wf": q.wfName, "realtime": q.realtime, "rank": q["rank"],
                  "pred": p,
                  "predrank": predranks, "angle": np.arccos((po.T @ qo) / (np.linalg.norm(qo) * np.linalg.norm(po))),
                  "dist": np.linalg.norm(po - qo),
                  "metric": np.linalg.norm((qo2 - po2))},
            columns=["nodeid", "task", "wf", "realtime", "pred", "predrank", "rank", "angle", "dist", "metric"])
    res3 = res3.append(insert, ignore_index=True)

In [243]:
res3

,nodeid,task,wf,realtime,pred,predrank,rank,angle,dist,metric
0,165,TRIMGALORE,nfcore/chipseq:1.2.2,15730.266667,17877.0,16,16,0.057276,53.40412,33.105891
1,166,TRIMGALORE,nfcore/chipseq:1.2.2,15871.033333,14486.0,12,17,0.057276,53.40412,33.105891
2,167,TRIMGALORE,nfcore/chipseq:1.2.2,15412.533333,10263.0,4,13,0.057276,53.40412,33.105891
3,168,TRIMGALORE,nfcore/chipseq:1.2.2,19916.533333,17877.0,16,22,0.057276,53.40412,33.105891
4,169,TRIMGALORE,nfcore/chipseq:1.2.2,20376.266667,19080.0,24,25,0.057276,53.40412,33.105891
...,...,...,...,...,...,...,...,...,...,...
57586,189,ApplyBQSR,nfcore/sarek:2.7.1,6785.133333,8000.0,16,27,0.060041,55.98214,47.655010
57587,190,ApplyBQSR,nfcore/sarek:2.7.1,6510.266667,5381.0,10,24,0.060041,55.98214,47.655010
57588,191,ApplyBQSR,nfcore/sarek:2.7.1,3831.833333,10415.0,26,10,0.060041,55.98214,47.655010
57589,192,ApplyBQSR,nfcore/sarek:2.7.1,3818.000000,4273.0,1,9,0.060041,55.98214,47.655010


In [244]:
r2_score(res4, res5)

-10.280530446496574

In [200]:
print(np.linalg.norm(res3["rank"] - res3["predrank"]))

2233.8439963435226


In [201]:
print(np.linalg.norm((res3["realtime"] - res3["pred"])))

4914678.00391379


In [202]:
print(np.average(res3.angle / 27))

0.00239919122103193


In [203]:
print(np.average(res3.dist / 27))

2.236913446913781


In [204]:
print(np.average(res3.metric / 27))

1.7040534467397166
